In [64]:
import pandas as pd
import numpy as np
import utm
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

print("Its alive!")

Its alive!


In [122]:
#Functions
def bin_groups(df, percent=.05):
    for col in df:
        if not pd.api.types.is_numeric_dtype(df[col]):
            for group, count in df[col].value_counts().iteritems():
                if count / len(df) < percent:
                    df.loc[df[col] == group, col] = 'Other'
    return df

def fs_variance(df, label="", p=0.8):
    from sklearn.feature_selection import VarianceThreshold
    import pandas as pd

    if label != "":
        X = df.drop(columns=[label])
    
    sel = VarianceThreshold(threshold=(p * (1 - p)))
    sel.fit_transform(X)

    # Add the label back in after removing poor features
    return df[sel.get_feature_names_out()].join(df[label])

def rule(row):
    long, lat = utm.to_latlon(row["LONG_UTM_X"], row["LAT_UTM_Y"], 12, 'T')
    return pd.Series({"long": long, "lat": lat})

def fit_crossvalidate_clf(df, label, k=10, r=5, repeat=True):
    import sklearn.linear_model as lm, pandas as pd, sklearn.ensemble as se, numpy as np
    from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
    from numpy import mean, std
    from sklearn import svm
    from sklearn import gaussian_process
    from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
    from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
    from sklearn import svm
    from sklearn.naive_bayes import CategoricalNB
    from xgboost import XGBClassifier
    from sklearn import preprocessing
    from sklearn.neural_network import MLPClassifier
    
    X = df.drop(columns=[label])
    y = df[label]
    
    if repeat:
        cv = RepeatedKFold(n_splits=k, n_repeats=r, random_state=12345)
    else:
        cv = KFold(n_splits=k, random_state=12345, shuffle=True)
    
    fit = {}    # Use this to store each of the fit metrics
    models = {} # Use this to store each of the models
    
    # Create the model objects
    model_log = lm.LogisticRegression(max_iter=100)
    model_logcv = lm.RidgeClassifier()
    model_sgd = lm.SGDClassifier(max_iter=1000, tol=1e-3)
    model_pa = lm.PassiveAggressiveClassifier(max_iter=1000, random_state=12345, tol=1e-3)
    model_per = lm.Perceptron(fit_intercept=False, max_iter=10, tol=None, shuffle=False)
    model_knn = KNeighborsClassifier(n_neighbors=3)
    model_svm = svm.SVC(decision_function_shape='ovo') # Remove the parameter for two-class model
    model_nb = CategoricalNB()
    model_bag = se.BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.5)
    model_ada = se.AdaBoostClassifier(n_estimators=100, random_state=12345)
    model_ext = se.ExtraTreesClassifier(n_estimators=100, random_state=12345)
    model_rf = se.RandomForestClassifier(n_estimators=10)
    model_hgb = se.HistGradientBoostingClassifier(max_iter=100)
    model_vot = se.VotingClassifier(estimators=[('lr', model_log), ('rf', model_ext), ('gnb', model_hgb)], voting='hard')
    model_gb = se.GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
    estimators = [('ridge', lm.RidgeCV()), ('lasso', lm.LassoCV(random_state=12345)), ('knr', KNeighborsRegressor(n_neighbors=20, metric='euclidean'))]
    final_estimator = se.GradientBoostingRegressor(n_estimators=25, subsample=0.5, min_samples_leaf=25, max_features=1, random_state=12345)
    model_st = se.StackingRegressor(estimators=estimators, final_estimator=final_estimator)
    model_xgb = XGBClassifier()
    model_nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=12345)
    
    # Fit a cross-validated R squared score and add it to the dict
    fit['Logistic'] = mean(cross_val_score(model_log, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
    fit['Ridge'] = mean(cross_val_score(model_logcv, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
    fit['SGD'] = mean(cross_val_score(model_sgd, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
    fit['PassiveAggressive'] = mean(cross_val_score(model_pa, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
    fit['Perceptron'] = mean(cross_val_score(model_per, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
    fit['KNN'] = mean(cross_val_score(model_knn, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
    fit['SVM'] = mean(cross_val_score(model_svm, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
    fit['NaiveBayes'] = mean(cross_val_score(model_nb, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
    fit['Bagging'] = mean(cross_val_score(model_bag, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
    fit['AdaBoost'] = mean(cross_val_score(model_ada, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
    fit['ExtraTrees'] = mean(cross_val_score(model_ext, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
    fit['RandomForest'] = mean(cross_val_score(model_rf, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
    fit['HistGradient'] = mean(cross_val_score(model_hgb, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
    fit['Voting'] = mean(cross_val_score(model_vot, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
    fit['GradBoost'] = mean(cross_val_score(model_gb, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
    fit['XGBoost'] = mean(cross_val_score(model_xgb, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
    fit['NeuralN'] = mean(cross_val_score(model_nn, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
    
    # Add the model to another dictionary; make sure the keys have the same names as the list above
    models['Logistic'] = model_log
    models['Ridge'] = model_logcv
    models['SGD'] = model_sgd
    models['PassiveAggressive'] = model_pa
    models['Perceptron'] = model_per
    models['KNN'] = model_knn
    models['SVM'] = model_svm
    models['NaiveBayes'] = model_nb
    models['Bagging'] = model_bag
    models['AdaBoost'] = model_ada
    models['ExtraTrees'] = model_ext
    models['RandomForest'] = model_rf
    models['HistGradient'] = model_hgb
    models['Voting'] = model_vot
    models['GradBoost'] = model_gb
    models['XGBoost'] = model_xgb
    models['NeuralN'] = model_nn
    
    # Add the fit dictionary to a new DataFrame, sort, extract the top row, use it to retrieve the model object from the models dictionary
    df_fit = pd.DataFrame({'Accuracy':fit})
    df_fit.sort_values(by=['Accuracy'], ascending=False, inplace=True)
    best_model = df_fit.index[0]
    print(df_fit)
    
    return models[best_model].fit(X, y)

In [66]:
drop_list = []
df = pd.read_csv("./Utah_Crash_Data_2020.csv")
df = df.drop([
"CRASH_ID",
# "CRASH_DATETIME",
# "CRASH_SEVERITY_ID",
# "ROUTE",                 //BIN
# "MILEPOINT",
# "LAT_UTM_Y",
# "LONG_UTM_X",
"MAIN_ROAD_NAME",
# "CITY",                  //BIN                 
# "COUNTY_NAME",           //BIN                  
# "WORK_ZONE_RELATED",                 
# "PEDESTRIAN_INVOLVED",               
# "BICYCLIST_INVOLVED",                
# "MOTORCYCLE_INVOLVED",               
# "IMPROPER_RESTRAINT",                
# "UNRESTRAINED",                      
# "DUI",                               
# "INTERSECTION_RELATED",              
# "WILD_ANIMAL_RELATED",               
# "DOMESTIC_ANIMAL_RELATED",           
# "OVERTURN_ROLLOVER",                 
# "COMMERCIAL_MOTOR_VEH_INVOLVED",     
# "TEENAGE_DRIVER_INVOLVED",           
# "OLDER_DRIVER_INVOLVED",             
# "NIGHT_DARK_CONDITION",              
# "SINGLE_VEHICLE",                    
# "DISTRACTED_DRIVING",                
# "DROWSY_DRIVING",                   
# "ROADWAY_DEPARTURE"
], axis=1)

df = df.dropna()

C:\Users\whyri\AppData\Local\Temp\ipykernel_17852\3141534565.py:2: DtypeWarning: Columns (0,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./Utah_Crash_Data_2020.csv")


In [67]:
df["CRASH_MONTH"] = pd.to_datetime(df["CRASH_DATETIME"]).dt.month
df["CRASH_WEEKDAY"] = pd.to_datetime(df["CRASH_DATETIME"]).dt.day_name()
df["CRASH_HOUR"] = pd.to_datetime(df["CRASH_DATETIME"]).dt.hour
df = df.drop(columns=["CRASH_DATETIME"])

In [68]:
for group, count in df["CITY"].value_counts().iteritems():
    if count / len(df) < .02:
        df.loc[df["CITY"] == group, "CITY"] = 'Other'

In [69]:
for group, count in df["COUNTY_NAME"].value_counts().iteritems():
    if count / len(df) < .02:
        df.loc[df["COUNTY_NAME"] == group, "COUNTY_NAME"] = 'Other'

In [70]:
for group, count in df["ROUTE"].value_counts().iteritems():
    if count / len(df) < .02:
        df.loc[df["ROUTE"] == group, "ROUTE"] = 'Other'

In [71]:
df.merge(df.apply(rule, axis=1), left_index=True, right_index=True)
df = df.drop(["LAT_UTM_Y","LONG_UTM_X"], axis=1)

In [72]:
df = pd.get_dummies(df, columns=[
    "ROUTE",
    "CRASH_MONTH",
    "CRASH_WEEKDAY",
    "CRASH_HOUR",
    "CITY",
    "COUNTY_NAME",
    "WORK_ZONE_RELATED",                 
    "PEDESTRIAN_INVOLVED",               
    "BICYCLIST_INVOLVED",                
    "MOTORCYCLE_INVOLVED",               
    "IMPROPER_RESTRAINT",                
    "UNRESTRAINED",                      
    "DUI",                               
    "INTERSECTION_RELATED",              
    "WILD_ANIMAL_RELATED",               
    "DOMESTIC_ANIMAL_RELATED",           
    "OVERTURN_ROLLOVER",                 
    "COMMERCIAL_MOTOR_VEH_INVOLVED",     
    "TEENAGE_DRIVER_INVOLVED",           
    "OLDER_DRIVER_INVOLVED",             
    "NIGHT_DARK_CONDITION",              
    "SINGLE_VEHICLE",                    
    "DISTRACTED_DRIVING",                
    "DROWSY_DRIVING",                   
    "ROADWAY_DEPARTURE"])
df.head()

,MILEPOINT,CRASH_SEVERITY_ID,ROUTE_15,ROUTE_68,ROUTE_80,ROUTE_89,ROUTE_Other,CRASH_MONTH_1,CRASH_MONTH_2,CRASH_MONTH_3,...,NIGHT_DARK_CONDITION_False,NIGHT_DARK_CONDITION_True,SINGLE_VEHICLE_False,SINGLE_VEHICLE_True,DISTRACTED_DRIVING_False,DISTRACTED_DRIVING_True,DROWSY_DRIVING_False,DROWSY_DRIVING_True,ROADWAY_DEPARTURE_False,ROADWAY_DEPARTURE_True
0,4.560,2.0,0,0,0,0,1,0,1,0,...,1,0,1,0,1,0,1,0,0,1
1,2.702,1.0,0,0,0,0,1,0,0,0,...,1,0,1,0,1,0,1,0,1,0
2,0.351,1.0,0,0,0,0,1,0,0,0,...,0,1,1,0,1,0,1,0,1,0
3,0.438,3.0,0,0,0,0,1,0,0,0,...,0,1,0,1,1,0,1,0,1,0
4,0.796,1.0,0,0,0,0,1,0,0,0,...,0,1,1,0,1,0,1,0,1,0


In [123]:
df_new = fs_variance(df, label="CRASH_SEVERITY_ID", p=.8)
# model = fit_crossvalidate_clf(df, "SaleCondition", 5, 2)
model = fit_crossvalidate_clf(df_new, "CRASH_SEVERITY_ID", 5, 2)

c:\Users\whyri\OneDrive\Desktop\intex_python\ml-venv2\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
